# Experiment Tracking
## Preprocessing

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb
from hyperopt import tpe, hp, Trials, STATUS_OK, fmin
from hyperopt.pyll import scope
import mlflow
import pickle

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
pd.options.mode.copy_on_write = True

In [3]:
def preprocessing(df):
    # compute target variables: trip duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # keep only trips of less than an hour
    df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

    # extract trip date components
    df['trip_month'] = pd.Categorical(df.lpep_pickup_datetime.dt.month)
    df['trip_dom'] = pd.Categorical(df.lpep_pickup_datetime.dt.day)
    df['trip_hour'] = pd.Categorical(df.lpep_pickup_datetime.dt.hour)

    # concatenate pickup and dropoff locations
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)

    return df

In [4]:
def one_hot_encoding(df, numerical, categorical, dv=None):
    df[categorical] = df[categorical].astype(str)
    df_dicts = df[categorical + numerical].to_dict(orient='records')

    if not dv:
        dv = DictVectorizer()
        dv.fit(df_dicts)

    X_train = dv.transform(df_dicts)

    return X_train, dv

In [5]:
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)

In [6]:
# read im the raw data
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [7]:
# data cleaning
df_train = preprocessing(df_train)
df_val = preprocessing(df_val)

In [8]:
# variable selection
categorical = ['PU_DO']
numerical = ['trip_distance']

In [9]:
# categorical feature encoding
X_train, dv = one_hot_encoding(df_train, numerical, categorical)
X_val, _ = one_hot_encoding(df_val, numerical, categorical, dv=dv)

In [10]:
target = 'duration'
y_train = df_train[target]
y_val = df_val[target]

In [11]:
with open('models/preprocessor.b', "wb") as f_out:
    pickle.dump(dv, f_out)

## Manual Logging

In [12]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/06/04 09:47:02 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1749023222199, experiment_id='1', last_update_time=1749023222199, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [13]:
mlflow.sklearn.autolog(disable=True)
mlflow.xgboost.autolog(disable=True)

### Lasso

In [14]:
alpha = .01
lr = Lasso(alpha=alpha)
train_model(lr, X_train, y_train)
y_preds = lr.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [15]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "Lasso")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.log_param("alpha", alpha)
    mlflow.sklearn.log_model(lr, "model")
    mlflow.log_metric("rmse", rmse)

2025/06/04 09:47:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Xgboost

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [17]:
def objective(params):
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50,
        verbose_eval=200
    )

    y_preds = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_preds)

    with mlflow.start_run():
        mlflow.set_tag("developer", "Armand Winant")

        mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
        mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
        mlflow.log_params(params)
        mlflow.xgboost.log_model(booster, "model")
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [18]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [19]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:11.05979                          
[200]	validation-rmse:6.52755                         
[400]	validation-rmse:6.47401                         
[600]	validation-rmse:6.44350                         
[800]	validation-rmse:6.42233                         
[999]	validation-rmse:6.41148                         
  0%|          | 0/50 [01:50<?, ?trial/s, best loss=?]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:49:28] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 09:49:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	validation-rmse:9.87885                                                      
[200]	validation-rmse:6.43034                                                    
[228]	validation-rmse:6.43581                                                    
  2%|▏         | 1/50 [03:47<1:46:08, 129.97s/trial, best loss: 6.41148433898481]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:51:25] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 09:51:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	validation-rmse:11.12086                                                     
[200]	validation-rmse:6.33151                                                    
[394]	validation-rmse:6.32218                                                    
  4%|▍         | 2/50 [06:00<1:38:19, 122.92s/trial, best loss: 6.41148433898481]

...

[0]	validation-rmse:10.73484                                                      
[200]	validation-rmse:6.42469                                                     
[400]	validation-rmse:6.38084                                                     
[600]	validation-rmse:6.36421                                                     
[800]	validation-rmse:6.35776                                                     
[999]	validation-rmse:6.35291                                                     
 96%|█████████▌| 48/50 [1:08:53<02:05, 62.57s/trial, best loss: 6.300646967506354]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:56:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 10:56:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	validation-rmse:10.29921                                                      
[200]	validation-rmse:6.34870                                                     
[400]	validation-rmse:6.31505                                                     
[548]	validation-rmse:6.31207                                                     
 98%|█████████▊| 49/50 [1:09:37<00:58, 58.74s/trial, best loss: 6.300646967506354]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:57:14] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2025/06/04 10:57:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 50/50 [1:09:46<00:00, 83.72s/trial, best loss: 6.300646967506354]


In [20]:
params = {
    'learning_rate': 0.23768760883453516,
    'max_depth': 32,
    'min_child_weight': 1.4255637138891217,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.20333941491253288,
    'reg_lambda': 0.014135076785365224,
    'seed': 42
}

In [21]:
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50,
    verbose_eval=200
)

y_preds = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_preds)

with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.log_params(params)
    mlflow.xgboost.log_model(booster, "model")
    mlflow.log_metric("rmse", rmse)

[0]	validation-rmse:10.35446
[200]	validation-rmse:6.35461
[400]	validation-rmse:6.31678
[581]	validation-rmse:6.30616


/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:57:59] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/06/04 10:58:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Random Forest

In [22]:
rf = RandomForestRegressor()
train_model(rf, X_train, y_train)
y_preds = rf.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [23]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "RandomForestRegressor")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(rf, "model")
    mlflow.log_metric("rmse", rmse)

2025/06/04 11:10:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Linear SVR

In [24]:
svr = LinearSVR(max_iter=10000000)
train_model(svr, X_train, y_train)
y_preds = svr.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [25]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "LinearSVR")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(svr, "model")
    mlflow.log_metric("rmse", rmse)

2025/06/04 11:53:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Gradient Boosting

In [26]:
gb = GradientBoostingRegressor()
train_model(gb, X_train, y_train)
y_preds = gb.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

In [27]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "GradientBoostingRegressor")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(gb, "model")
    mlflow.log_metric("rmse", rmse)

2025/06/04 11:53:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Extra Trees

In [28]:
et = ExtraTreesRegressor()
train_model(et, X_train, y_train)
y_preds = et.predict(X_val)
rmse = root_mean_squared_error(y_val, y_preds)

KeyboardInterrupt: 

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    mlflow.log_param("training-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "ExtraTreesRegressor")

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.sklearn.log_model(et, "model")
    mlflow.log_metric("rmse", rmse)